In [1]:
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xview.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xfixed.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xtensor.hpp>
#include <xtensor/xadapt.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xaxis_slice_iterator.hpp>
#include <xtensor/xaxis_iterator.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xindex_view.hpp>
#include <xtensor/xchunked_array.hpp>
#include <xtensor/xfixed.hpp>
#include <cstddef>
#include <vector>
#include <xtensor/xarray.hpp>
#include <xtensor/xadapt.hpp>
#include <istream>
#include <fstream>
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xcsv.hpp>
#include <iostream>
#include <fstream>
#include <functional>
#include <cstdio>
#include <xtensor/xcsv.hpp>
#include <iostream>
#include <chrono>
#include <unistd.h>

using namespace xt;
using namespace std;

#include <chrono>
using namespace std::chrono;

In [2]:
double current_time(){
    auto current_time = std::chrono::system_clock::now();
    auto duration_in_seconds = std::chrono::duration<double>(current_time.time_since_epoch());
    double num_seconds = duration_in_seconds.count();
    return num_seconds;
}

# Lecture dataset

In [29]:
using namespace std;

ofstream outfile;
outfile.open("../out/LOG_dbscan_cpp_lecture_dataset_D2_R12_m4_e2_euclidean_distance.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/lecture_dataset.txt");
auto data = xt::load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << "," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

## Similarity

In [30]:
float euclidean_distance(const xarray<double>& a, const xarray<double>& b){ 
    return (float) sqrt(sum(pow((a - b), 2))(0));
}

## Clusterization

In [31]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [32]:
auto dbscan(const xarray<float>& X, float epsilon, int minPts)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;

    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    
    auto get_neighbors = [&] (int current_index) {
        list<int> neighbor_indices = list<int>();
        for (int neighbor_index : arange(0, n)){
            
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," <<
                current_index << ",1," << endl;
            
            if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= (float) epsilon){
                neighbor_indices.push_back((int) neighbor_index);
            };
        }
        return neighbor_indices;
    };
    
    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        auto start =  chrono::high_resolution_clock::now();
        list<int> neighbor_indices = get_neighbors(current_index);
        auto end = chrono::high_resolution_clock::now();
        std::chrono::duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",Eps_time," <<
                current_index << "," << elapsed.count() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",|Eps_neighbors|," <<
                current_index << "," << (int) neighbor_indices.size() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",Eps_neighbor_id," <<
                current_index << ",,";
        for(auto& i : neighbor_indices) {outfile << i << ";"; }
        outfile << endl;

        if ((int) neighbor_indices.size() >= minPts){
            state(current_index) = CLUSTERED;
            cluster(current_index) = cluster_id;
            for (auto& neighbor_index : neighbor_indices) {
                if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
                    state(neighbor_index) = CLUSTERED;
                    cluster(neighbor_index) = cluster_id;
                    search(neighbor_index);
                };
            };
        } else {
            state(current_index) = VISITED;
        };
    };

    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
}

In [33]:
dbscan(X, 2, 4);

## Save output to csv

In [34]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscan_cpp_lecture_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << ',' << endl;
outfile.close();

# Compound dataset

In [9]:
ofstream outfile;
outfile.open("../out/LOG_dbscan_cpp_compound_dataset_D2_R399_m3_e0.7_euclidean_distance.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/compound_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << "," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [10]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [11]:
auto dbscan(const xarray<float>& X, float epsilon, int minPts)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;

    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    
    auto get_neighbors = [&] (int current_index) {
        list<int> neighbor_indices = list<int>();
        for (int neighbor_index : arange(0, n)){
            
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," <<
                current_index << ",1," << endl;
            
            if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= (float) epsilon){
                neighbor_indices.push_back((int) neighbor_index);
            };
        }
        return neighbor_indices;
    };
    
    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        auto start =  chrono::high_resolution_clock::now();
        list<int> neighbor_indices = get_neighbors(current_index);
        auto end = chrono::high_resolution_clock::now();
        std::chrono::duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",Eps_time," <<
                current_index << "," << elapsed.count() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",|Eps_neighbors|," <<
                current_index << "," << (int) neighbor_indices.size() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",Eps_neighbor_id," <<
                current_index << ",,";
        for(auto& i : neighbor_indices) {outfile << i << ";"; }
        outfile << endl;

        if ((int) neighbor_indices.size() >= minPts){
            state(current_index) = CLUSTERED;
            cluster(current_index) = cluster_id;
            for (auto& neighbor_index : neighbor_indices) {
                if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
                    state(neighbor_index) = CLUSTERED;
                    cluster(neighbor_index) = cluster_id;
                    search(neighbor_index);
                };
            };
        } else {
            state(current_index) = VISITED;
        };
    };

    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
}

In [12]:
dbscan(X, 0.7, 3);

## Save output to csv

In [13]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscan_cpp_compound_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# Jain dataset

In [14]:
ofstream outfile;
outfile.open("../out/LOG_dbscan_cpp_jain_dataset_D2_R373_m14_e2_euclidean_distance.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/jain_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << "," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [15]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [16]:
auto dbscan(const xarray<float>& X, float epsilon, int minPts)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;

    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    
    auto get_neighbors = [&] (int current_index) {
        list<int> neighbor_indices = list<int>();
        for (int neighbor_index : arange(0, n)){
            
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," <<
                current_index << ",1," << endl;
            
            if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= (float) epsilon){
                neighbor_indices.push_back((int) neighbor_index);
            };
        }
        return neighbor_indices;
    };
    
    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        auto start =  chrono::high_resolution_clock::now();
        list<int> neighbor_indices = get_neighbors(current_index);
        auto end = chrono::high_resolution_clock::now();
        std::chrono::duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",Eps_time," <<
                current_index << "," << elapsed.count() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",|Eps_neighbors|," <<
                current_index << "," << (int) neighbor_indices.size() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",Eps_neighbor_id," <<
                current_index << ",,";
        for(auto& i : neighbor_indices) {outfile << i << ";"; }
        outfile << endl;

        if ((int) neighbor_indices.size() >= minPts){
            state(current_index) = CLUSTERED;
            cluster(current_index) = cluster_id;
            for (auto& neighbor_index : neighbor_indices) {
                if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
                    state(neighbor_index) = CLUSTERED;
                    cluster(neighbor_index) = cluster_id;
                    search(neighbor_index);
                };
            };
        } else {
            state(current_index) = VISITED;
        };
    };

    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
}

In [17]:
dbscan(X, 2, 14);

## Save output to csv

In [18]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscan_cpp_jain_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# Flame dataset

In [35]:
ofstream outfile;
outfile.open("../out/LOG_dbscan_cpp_flame_dataset_D2_R240_m23_e2.067_euclidean_distance.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/flame_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << "," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [36]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [37]:
auto dbscan(const xarray<float>& X, float epsilon, int minPts)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;

    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    
    auto get_neighbors = [&] (int current_index) {
        list<int> neighbor_indices = list<int>();
        for (int neighbor_index : arange(0, n)){
            
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," <<
                current_index << ",1," << endl;
            
            if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= (float) epsilon){
                neighbor_indices.push_back((int) neighbor_index);
            };
        }
        return neighbor_indices;
    };
    
    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        auto start =  chrono::high_resolution_clock::now();
        list<int> neighbor_indices = get_neighbors(current_index);
        auto end = chrono::high_resolution_clock::now();
        std::chrono::duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",Eps_time," <<
                current_index << "," << elapsed.count() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",|Eps_neighbors|," <<
                current_index << "," << (int) neighbor_indices.size() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",Eps_neighbor_id," <<
                current_index << ",,";
        for(auto& i : neighbor_indices) {outfile << i << ";"; }
        outfile << endl;

        if ((int) neighbor_indices.size() >= minPts){
            state(current_index) = CLUSTERED;
            cluster(current_index) = cluster_id;
            for (auto& neighbor_index : neighbor_indices) {
                if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
                    state(neighbor_index) = CLUSTERED;
                    cluster(neighbor_index) = cluster_id;
                    search(neighbor_index);
                };
            };
        } else {
            state(current_index) = VISITED;
        };
    };

    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
}

In [38]:
dbscan(X, 2.067, 23);

## Save output to csv

In [39]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscan_cpp_flame_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# R15 dataset

In [24]:
ofstream outfile;
outfile.open("../out/LOG_dbscan_cpp_R15_dataset_D2_R600_m2_e0.3_euclidean_distance.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/R15_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << "," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [25]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [26]:
auto dbscan(const xarray<float>& X, float epsilon, int minPts)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;

    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    
    auto get_neighbors = [&] (int current_index) {
        list<int> neighbor_indices = list<int>();
        for (int neighbor_index : arange(0, n)){
            
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," <<
                current_index << ",1," << endl;
            
            if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= (float) epsilon){
                neighbor_indices.push_back((int) neighbor_index);
            };
        }
        return neighbor_indices;
    };
    
    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        auto start =  chrono::high_resolution_clock::now();
        list<int> neighbor_indices = get_neighbors(current_index);
        auto end = chrono::high_resolution_clock::now();
        std::chrono::duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",Eps_time," <<
                current_index << "," << elapsed.count() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",|Eps_neighbors|," <<
                current_index << "," << (int) neighbor_indices.size() << ',' << endl;
        outfile << std::setprecision (16) << current_time() << ",Eps_neighbor_id," <<
                current_index << ",,";
        for(auto& i : neighbor_indices) {outfile << i << ";"; }
        outfile << endl;

        if ((int) neighbor_indices.size() >= minPts){
            state(current_index) = CLUSTERED;
            cluster(current_index) = cluster_id;
            for (auto& neighbor_index : neighbor_indices) {
                if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
                    state(neighbor_index) = CLUSTERED;
                    cluster(neighbor_index) = cluster_id;
                    search(neighbor_index);
                };
            };
        } else {
            state(current_index) = VISITED;
        };
    };

    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
}

In [27]:
dbscan(X, 0.3, 2);

## Save output to csv

In [28]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscan_cpp_R15_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();